# Requests

Стандартная библиотека python включает в себя все необходимое для запросов по http и https:
* модуль urllib
* модуль httplib
* модуль socket
* модуль ssl

Рассмотрим популярный модуль **requests**, позволяющая осуществлять запросы http и https значительно проще.

В качестве сайта, к которому будут совершаться различные запросы, был выбран https://httpbin.org/. Он обрабатывает все возможные запросы и выдает подробную отладочную информацию в ответе.

Для начала сделаем простой [*get-запрос*](https://www.elma-bpm.ru/KB/help/RU/Platform/content/Designer_SOA_Web_Get_index.html) и рассмотрим полученный ответ:

In [ ]:
import requests

In [ ]:
response = requests.get('https://httpbin.org/get')  # Записываем ответ на наш запрос в переменную response

Посмотрим содрежимое объекта response.

In [ ]:
print('Объект Response:', response)

Код 200 означает, что запрос успешно выполнен. Можно запросить только код ответа: 

In [ ]:
print('Код ответа:', response.status_code)

print('Запрос успешен?', response.ok)

В блоке ниже ответ сначала в виде [байтовой строки](https://pythononline.ru/osnovy/bytes-python), а затем в виде [JSON-нотации](https://www.json.org/json-ru.html):

In [ ]:
print('Содержимое ответа:')
print(response.content)  
print('\nТекст ответа (строка):')
print(response.text)

In [ ]:
print('Кодировка ответа (используется при распознавании .content в .text):', response.encoding)
print('Заголовки ответа (массив):', response.headers)

Если текст ответа - данные в формате [*json*](https://www.json.org/json-ru.html), то метод .json позволит получить данные в виде python словаря:

In [ ]:
parsed_json = response.json()  # Получение ответа в формате json
print('Результат .json() - набор данных в стандартных структурах python:', type(parsed_json))

print('Отправленный заголовок User-Agent:', parsed_json['headers']['User-Agent'])

Рассмотрим дополительные параметры метода `.get()`:

In [ ]:
response = requests.get(
    'https://httpbin.org/get',
    params={'My Param': 'My value', '?!&': '*&@!', 'array': [1, 2, 3]},  # Задаем параметры
    headers={'X-My-Header': 'SomeValue'},  # Задаем заголовки
    auth=('Login', 'Password'),  # Передаем данные для авторизации
    cookies={'My Cookie': 'Cookie Value'}  # Передаем cookie (если это не первый запрос к адресу)
)
print(response.text)  # Печатаем ответ

Аргумент *params* конструирует параметры запроса (которые передаются после "?" в *url*) из словаря. Все значения приобретают правильную кодировку, можно использовать любые символы. Как видите, сервер корректно распознал наши аргументы (*args*). Для передачи данные были преобразованы в нечитаемый *urlencoding* (можно увидеть в *url*).

С помощью аргумента headers мы можем передать словарь с произвольными http заголовками, которые нужно отправить на сервер. В тексте ответа можно заметить отправленный заголовок *X-My-Header*.

Аргумент *auth* используется для создания заголовка Authorization в правильном формате (закодированая base64 строка).

Модуль **base64** предоставляет функции для кодирования двоичных данных в  символы ASCII и декодирования таких кодировок обратно в двоичные данные.

In [ ]:
import base64  

# Вычленяем из ответа связку логин:пароль и преобразуем в тип данных bytes
base64.b64decode(response.json()['headers']['Authorization'].split()[1])

Аргумент cookies позволяет вручную указывать передаваемые Cookie. Для более тонкой настройки (время жизни, домены) можно использовать `requests.cookies.RequestsCookieJar`, но удобнее всего использовать сессии.

Все предыдущие примеры были рассмотрены на основе запроса `GET`. При использовании запроса [`POST`](https://htmlacademy.ru/blog/boost/frontend/get-vs-post) (используется при отправке форм, файлов) становятся доступными еще 3 параметра:
*   `data`
*   `json`
*   `files`

`data` позволяет отправлять данные в виде набора байтов.

In [ ]:
response = requests.post(
    'https://httpbin.org/post',  # Адрес запроса
    data='Data in post request body',  # Данные запроса
    params={'Data in': 'url parameters'}  # Можно посылать независимо от data-post
)
print(response.text)

Если в `data` передать словарь, то данные будут отправлены в формате html-формы.

In [ ]:
response = requests.post(
    'https://httpbin.org/post',
    data={'field1': 'Hello', 'field2': 'World'}
)
print(response.text)

Параметр `json` позволяет использовать в теле запроса данные в формате json.

In [ ]:
response = requests.post(
    'https://httpbin.org/post',
    json={'field1': 'Hello', 'field2': 'World'},  # Передаем данные в формате json
)
print(response.text)

`files` позволяет отправлять файлы в post запросе (в формате multipart/form-data):

In [ ]:
# Создаем файл для отправки
with open('request_test.txt', 'w') as f:
  f.write('some data')

In [ ]:
response = requests.post(
    'https://httpbin.org/post',
    files={'request_test.txt': ('request_test.txt', open('request_test.txt'))}
)
print(response.text)

## Дополнительные функции

Подробно про остальные функции можно прочитать в [документации](http://docs.python-requests.org/en/latest/api/). 

Для удобной работы с [Cookies](https://ssl.com.ua/blog/what-are-cookies/) можно воспользоваться сессией:

In [ ]:
with requests.Session() as sess:  # Важно использовать контекст with, чтобы убедиться в выполнении кода
    sess.get('https://httpbin.org/cookies/set/MyCookie/MyValue')
    print(sess.get('https://httpbin.org/cookies').text)

Если не использовать общую сессию для этих двух запросов, то значение *cookie* не отправляется во втором запросе:

In [ ]:
print('1 запрос:')
response = requests.get('https://httpbin.org/cookies/set/MyCookie/MyValue')

print(response.headers)

print('2 запрос:')
print(requests.get('https://httpbin.org/cookies').text)

На этом основные функции requests подходят к концу. Дополнительные функции:

Рассмотренные методы `.get` и `.post` на самом деле являются обертками над основным методом модуля [requests.request](http://docs.python-requests.org/en/latest/api/#requests.request).

In [ ]:
print(requests.request('GET', 'https://httpbin.org/get').text)

Все рассмотренные параметры передаются напрямую в метод `.request`.

В редких случаях кроме `GET` и `POST` требуется использовать другие методы HTTP запросов: `OPTIONS`/`PUT`/`PATCH`/`DELETE`/`HEAD`. Для этого можно использовать `requests.request('<метод>')` или `requests.<метод>`.

In [ ]:
print(requests.delete('https://httpbin.org/anything').text)

В случае получения большого запроса или потоковой обработки данных можно запрашивать небольшие кусочки данных и читать их без обработки с помощью `response.raw`.

In [ ]:
response = requests.get('https://httpbin.org/bytes/50', stream=True)  # Данные передаются поточно
data = response.raw.read(10)  # Считываем данные кусочно, без обработки
while data:
    print('Кусочек данных:', data)
    data = response.raw.read(10)


Альтернативный способ частичного скачивания, используя итераторы. В данном случае мы еще сохраняем данные в файл.

In [ ]:
response = requests.get('https://httpbin.org/bytes/30', stream=True)
with open('random_data', 'wb') as file:
    for chunk in response.iter_content(10):
        print('Кусочек данных:', chunk)
        file.write(chunk)

Для наилучшей производительности следует использовать размер куска в 2048/4096/8192 байт.

Для ограничения запроса по времени можно использовать параметр timeout:

In [ ]:
requests.get('https://httpbin.org/delay/3', timeout=0.5)